In [ ]:
from matplotlib import pyplot as plt  # type: ignore
from colorhash import ColorHash  # type: ignore
import sqlalchemy as sq
import geopandas as gpd  # type: ignore
import pandas as pd  # type: ignore
from dotenv import load_dotenv
import os, sys

sys.path.append("../")
from Shared.DataService import DataService  # type: ignore

In [ ]:
TABLENAME = "labeled_soil"

load_dotenv()
PG_DB = os.getenv("POSTGRES_DB")
PG_ADDR = os.getenv("POSTGRES_ADDR")
PG_PORT = os.getenv("POSTGRES_PORT")
PG_USER = os.getenv("POSTGRES_USER")
PG_PW = os.getenv("POSTGRES_PW")

In [ ]:
db = DataService(PG_DB, PG_ADDR, PG_PORT, PG_USER, PG_PW)
conn = db.connect()

In [ ]:
# load the boundaries for the agriculture regions
query = sq.text("select cr_num, car_uid, geometry FROM public.census_ag_regions")  # type: ignore
agRegions = gpd.GeoDataFrame.from_postgis(query, conn, crs="EPSG:3347", geom_col="geometry")  # type: ignore

In [ ]:
# load the polygon components
query = pd.read_sql_query("select soil_id, poly_id FROM public.soil_components", conn)
components = pd.DataFrame(query, columns=["soil_id", "poly_id"])
components = components.groupby("poly_id")["soil_id"].apply(set)

In [ ]:
# load the boundaries for the polygons
query = sq.text("select poly_id, geometry FROM public.soil_geometry")
soilRegions = gpd.GeoDataFrame.from_postgis(
    query, conn, crs="EPSG:3347", geom_col="geometry"
)

In [ ]:
# merge the components and the soil boundaries into one
regionsAndComponents = soilRegions.merge(components, on="poly_id")

In [ ]:
# join the the newly merged soil dataframe with the agriculture boundaries
soil = gpd.sjoin(regionsAndComponents, agRegions, how="left", predicate="intersects")
soil = soil[soil["cr_num"].notna()]  # Take rows that are valid numbers
soil.drop(columns=["index_right"], inplace=True)
soil = soil.reset_index()

In [ ]:
soil.drop(columns=["index"], inplace=True)
soil = soil.reset_index()

In [ ]:
soil.plot()

In [ ]:
# reload each set as a list by ordering their contents
listOfSoilIDs = []

for index, row in soil.iterrows():
    listOfSoilIDs.append(sorted(row["soil_id"]))

soil["soil_id"] = listOfSoilIDs

In [ ]:
# hash each set of soil to create a consistant color scheme for similar soil typings
colors = []

for index in range(len(soil.index)):
    value = soil.iloc[index]["poly_id"]
    colors.append(ColorHash(str(value)).hex)

soil["color"] = colors

In [ ]:
minx, miny, maxx, maxy = soil.total_bounds
fig, ax = plt.subplots(figsize=(13, 13))
ax.set_ylim(miny, maxy)
soil.plot(ax=ax, color=soil["color"])
agRegions.geometry.boundary.plot(color=None, edgecolor="k", linewidth=2, ax=ax)

plt.show()

In [ ]:
# prepare the data for database storage
labeledSoil = pd.DataFrame(soil.drop(columns=["geometry", "color"]))

labeledSoil.rename(columns={labeledSoil.columns[0]: "id"}, inplace=True)
labeledSoil.rename(columns={labeledSoil.columns[2]: "soil_ids"}, inplace=True)

In [ ]:
labeledSoil

In [ ]:
labeledSoil.to_sql(TABLENAME, conn, schema="public", if_exists="append", index=False)

In [ ]:
db.cleanup()